In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [2]:
working_directory = os.getcwd()
#data_folder =  os.path.join(working_directory, "data")
results_folder = os.path.join(working_directory, "results")

tract_filename = os.path.join(working_directory, r"tl_2019_49_tract\tl_2019_49_tract.shp")

tazcentroids_filename = working_directory + r"\TAZ\USTM_TAZ_2021_09_22_Centroids.shp"

In [3]:
sdf_tract = pd.DataFrame.spatial.from_featureclass(tract_filename)

sdf_tract.head()

FID STATEFP COUNTYFP TRACTCE        GEOID     NAME              NAMELSAD  \
0    0      49      049  002101  49049002101    21.01    Census Tract 21.01   
1    1      49      049  002102  49049002102    21.02    Census Tract 21.02   
2    2      49      035  113101  49035113101  1131.01  Census Tract 1131.01   
3    3      49      005  001201  49005001201    12.01    Census Tract 12.01   
4    4      49      035  113102  49035113102  1131.02  Census Tract 1131.02   

   MTFCC FUNCSTAT       ALAND  AWATER     INTPTLAT      INTPTLON  \
0  G5020        S   2118017.0     0.0  +40.2535857  -111.6879803   
1  G5020        S   1745061.0     0.0  +40.2472341  -111.6810009   
2  G5020        S   4797256.0     0.0  +40.5968191  -111.9931221   
3  G5020        S  16624874.0     0.0  +41.6713238  -111.8144872   
4  G5020        S   2522097.0     0.0  +40.5852938  -111.9838257   

                                               SHAPE  
0  {"rings": [[[-111.700557, 40.256361], [-111.70...  
1  {"rings": [[[-111.695308, 40.244827], [-111.69...  
2  {"rings": [[[-112.005461, 40.602444], [-112.00...  
3  {"rings": [[[-111.835015, 41.699234], [-111.83...  
4  {"rings": [[[-112.004958, 40.580537], [-112.00...

In [4]:
#verify that shape exists and sdf was created correctly and get spatial reference
print(sdf_tract.iloc[0].SHAPE.spatialReference)

{'wkid': 4269, 'latestWkid': 4269}


In [5]:
#change projection to 26912 to match TAZ Centroids
sdf_tract.spatial.project(26912)

True

In [6]:
#verify that shape exists and sdf was created correctly and get spatial reference
print(sdf_tract.iloc[0].SHAPE.spatialReference)

{'wkid': 26912, 'latestWkid': 26912}


In [7]:
sdf_tazcentroids = pd.DataFrame.spatial.from_featureclass(tazcentroids_filename)

sdf_tazcentroids.head()

FID  TAZID  SA_TAZID  CO_IDX  CO_TAZID  SUBAREAID       ACRES    DEVACRES  \
0    0     43         3       3      1003          0  440.680556  440.680556   
1    1     45         5       5      1005          0  245.424579  245.424579   
2    2     46         6       6      1006          0  675.848293  675.848293   
3    3     55        15      15      1015          0  550.767119  534.244106   
4    4     50        10      10      1010          0  176.207617  176.207617   

   DEVPBLEPCT              X  ...  TAZID_V21B  SATAZ_V21B  COTAZ_V21B  \
0        1.00  357308.618265  ...          43           0        1013   
1        1.00  356023.333451  ...          45           0        1015   
2        1.00  357530.306637  ...          46           0        1016   
3        0.97  356277.254346  ...          55           0        1025   
4        1.00  355998.119155  ...          50           0        1020   

  SUBID_V21B TAZID_V21A  SATAZ_V21A COTAZ_V21A  SUBID_V21A ORIG_FID  \
0          0         43           0       1013           0        0   
1          0         45           0       1015           0        1   
2          0         46           0       1016           0        2   
3          0         55           0       1025           0        3   
4          0         50           0       1020           0        4   

                                               SHAPE  
0  {"x": 357308.61828827206, "y": 4240407.8890121...  
1  {"x": 356023.33349711914, "y": 4239022.5529349...  
2  {"x": 357530.3066365821, "y": 4239070.37353264...  
3  {"x": 356277.2543912772, "y": 4234918.19385224...  
4  {"x": 355998.11920056865, "y": 4237044.8358770...  

[5 rows x 34 columns]

In [8]:
#verify that shape exists and sdf was created correctly and get spatial reference
print(sdf_tazcentroids.iloc[0].SHAPE)

{'x': 357308.61828827206, 'y': 4240407.889012115, 'spatialReference': {'wkid': 26912, 'latestWkid': 26912}}


In [9]:
#spatial join tract centroid data to zip code data
sdf_tazcentroids_tract_join = sdf_tazcentroids.spatial.join(sdf_tract)

sdf_tazcentroids_tract_join.head()

FID_left  TAZID  SA_TAZID  CO_IDX  CO_TAZID  SUBAREAID       ACRES  \
0         0     43         3       3      1003          0  440.680556   
1         1     45         5       5      1005          0  245.424579   
2         2     46         6       6      1006          0  675.848293   
3         3     55        15      15      1015          0  550.767119   
4         4     50        10      10      1010          0  176.207617   

     DEVACRES  DEVPBLEPCT              X  ...  TRACTCE        GEOID  NAME  \
0  440.680556        1.00  357308.618265  ...   100100  49001100100  1001   
1  245.424579        1.00  356023.333451  ...   100100  49001100100  1001   
2  675.848293        1.00  357530.306637  ...   100100  49001100100  1001   
3  534.244106        0.97  356277.254346  ...   100100  49001100100  1001   
4  176.207617        1.00  355998.119155  ...   100100  49001100100  1001   

            NAMELSAD  MTFCC  FUNCSTAT         ALAND    AWATER     INTPTLAT  \
0  Census Tract 1001  G5020         S  1.353681e+09  830378.0  +38.3450604   
1  Census Tract 1001  G5020         S  1.353681e+09  830378.0  +38.3450604   
2  Census Tract 1001  G5020         S  1.353681e+09  830378.0  +38.3450604   
3  Census Tract 1001  G5020         S  1.353681e+09  830378.0  +38.3450604   
4  Census Tract 1001  G5020         S  1.353681e+09  830378.0  +38.3450604   

       INTPTLON  
0  -112.5833973  
1  -112.5833973  
2  -112.5833973  
3  -112.5833973  
4  -112.5833973  

[5 rows x 48 columns]

In [10]:
df_tazcentroids_tract = sdf_tazcentroids_tract_join[['CO_TAZID','GEOID']].copy()
df_tazcentroids_tract

CO_TAZID        GEOID
0         1003  49001100100
1         1005  49001100100
2         1006  49001100100
3         1015  49001100100
4         1010  49001100100
...        ...          ...
9810     21219  49021110600
9811     21354  49021110600
9812     21353  49021110600
9813     21356  49021110600
9814     21355  49021110600

[9815 rows x 2 columns]

In [11]:
df_tazcentroids_tract.to_csv(os.path.join(results_folder, r"TAZ_CensusTract.csv"))